# Earthmover's Distance (Wasserstein Distance) using Linear Programming
This Jupyter notebook is a simple implementation of the optimization problem of the Earthmover's distance a.k.a Wasserstein distance. The problem is solved using the linear programming solver from the scipy library.

The notebook is divided into the following sections:

Author: [Sachin Motwani](https://www.linkedin.com/in/sachin-motwani/)

## Example 1
Earthmover's Distance visualisaiton between two continuous distributions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import wasserstein_distance

# Generate sample data
np.random.seed(42)
pr_samples = np.random.normal(0, 1, 1000)
pg_samples = np.random.normal(2, 1, 1000)

# Compute the Wasserstein distance
distance = wasserstein_distance(pr_samples, pg_samples)
print(f"Wasserstein Distance: {distance}")

# Visualization
plt.hist(pr_samples, bins=30, alpha=0.5, label='P_r ($\mu=0, \sigma^2=1$)')
plt.hist(pg_samples, bins=30, alpha=0.5, label='P_g ($\mu=2, \sigma^2=1$)')
plt.legend()
plt.title('Distributions and Wasserstein Distance')
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()


## Example 2

Earthmover's Distance optimization using LP Solver

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog

# Function to choose text color for contrast
def get_text_color(cell_value, max_value):
    return "black" if cell_value < max_value / 2 else "white"

# Data
positions_Pr = np.array([0, 1, 2])
masses_Pr = np.array([2, 0, 2])
positions_Ptheta = np.array([5, 6, 7])
masses_Ptheta = np.array([1, 0, 2])

# Plot the orginal distributions
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].bar(positions_Pr, masses_Pr, width=0.3, color='blue', label='P_r')
ax[0].set_title('Initial Distribution P_r')
ax[0].set_xlabel('Position')
ax[0].set_ylabel('Mass')
ax[1].bar(positions_Ptheta, masses_Ptheta, width=0.3, color='green', label='P_theta')
ax[1].set_title('Final Distribution P_theta')
ax[1].set_xlabel('Position')
ax[1].set_ylabel('Mass')
plt.show()


# Normalize the masses so that they sum up to the same total mass
total_mass_Pr = np.sum(masses_Pr)
total_mass_Ptheta = np.sum(masses_Ptheta)
masses_Pr = masses_Pr / total_mass_Pr
masses_Ptheta = masses_Ptheta / total_mass_Ptheta
# print("Normalized Masses for Pr: ",masses_Pr,"\nNormalized Masses for Ptheta: ",masses_Ptheta)

# Create the cost matrix based on the distances between positions
cost_matrix = np.abs(positions_Pr[:, None] - positions_Ptheta[None, :])

# Plot the cost matrix
# Define the extent of the matrix so that the cells are centered from 0 to the number of positions
extent = [min(positions_Ptheta)-0.5, max(positions_Ptheta)+0.5, min(positions_Pr)-0.5, max(positions_Pr)+0.5]
fig, ax = plt.subplots(figsize=(6, 6))
im = ax.imshow(cost_matrix, cmap='Blues', origin='lower', extent=extent)
ax.set_title('Cost Matrix')
ax.set_xlabel('Ptheta')
ax.set_ylabel('Pr')
ax.figure.colorbar(im, ax=ax, label='Cost')
# Add text annotations
max_value_cost = np.max(cost_matrix)
for i in range(len(positions_Pr)):
    for j in range(len(positions_Ptheta)):
        ax.text(j + min(positions_Ptheta), i + min(positions_Pr), 
                f"{cost_matrix[i, j]:.1f}",
                ha="center", va="center", 
                color=get_text_color(cost_matrix[i, j], max_value_cost))
plt.show()


# Linear programming to compute the optimal transport plan
# Number of sources and targets
num_sources = len(positions_Pr)
num_targets = len(positions_Ptheta)
# Flatten the cost matrix to get the cost for each potential path
c = cost_matrix.flatten()
# Initialize the equality constraint matrix with zeros
A_eq = np.zeros((num_sources + num_targets, num_sources * num_targets))
# print("Equality Constraints: ",A_eq)
# Set up the equality constraints for the sources
for i in range(num_sources):
    A_eq[i, i * num_targets:(i + 1) * num_targets] = 1
    # print("For i = ", i, " the Equality Constraints is ",A_eq)
# Set up the equality constraints for the targets
for j in range(num_targets):
    A_eq[num_sources + j, j::num_targets] = 1
    # print("For j = ", j, " the Equality Constraints is ",A_eq)
# Concatenate the source masses and target masses for the constraints
b_eq = np.concatenate([masses_Pr, masses_Ptheta])
# Define the bounds for each variable, which is the transport amount
bounds = [(0, None)] * len(c)
# 0  indicates that the variables cannot be negative [Real valued non-negative]
# None indicates that there is no upper bound on the variables



# Solve the linear programming problem
result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')
# 'c' is the vectorised (flattened) cost matrix
# 'A_eq' is the equality constraint matrix [LHS of the equality constraints] 
# 'b_eq' is the equality constraint vector [RHS of the equality constraints]
# 'bounds' is the bounds for each variable
# 'highs' is a high performance linear programming solver
# print("Cost Matrix: ",c)
# print("Equality Constraints: ",A_eq)
# print("Equality Constraints: ",b_eq)
# print("Bounds: ",bounds)
# print("Result: ",result)

# Reshape the result to get the optimal transport plan
optimal_transport_plan = result.x.reshape(num_sources, num_targets)
# print("Optimal Transport Plan: ",optimal_transport_plan)

# Plot the heat map of the optimal transport plan
fig, ax = plt.subplots(figsize=(6, 6))
im = ax.imshow(optimal_transport_plan, cmap='Blues', origin='lower', extent=extent)
ax.set_title('Optimal Transport Plan')
ax.set_xlabel('Ptheta')
ax.set_ylabel('Pr')
ax.figure.colorbar(im, ax=ax, label='Mass to be moved from Pr to Ptheta')
# Add text annotations
max_value_optimal = np.max(optimal_transport_plan)
for i in range(len(positions_Pr)):
    for j in range(len(positions_Ptheta)):
        ax.text(j + min(positions_Ptheta), i + min(positions_Pr), 
                f"{optimal_transport_plan[i, j]:.2f}",
                ha="center", va="center", 
                color=get_text_color(optimal_transport_plan[i, j], max_value_optimal))
plt.show()


# Calculate the Wasserstein distance
wasserstein_distance = np.sum(optimal_transport_plan * cost_matrix)



## Plot the initial and final distributions and the optimal transport plan with thicker bars
fig, ax = plt.subplots(1, 3, figsize=(18, 6))

# Plot the initial distribution P_r with increased bar width
ax[0].bar(positions_Pr, masses_Pr, width=0.3, color='blue', label='P_r')
ax[0].set_title('Initial Distribution P_r')
ax[0].set_xlabel('Position')
ax[0].set_ylabel('Normalized Mass')

# Plot the optimal transport plan with arrows
for i in range(len(positions_Pr)):
    for j in range(len(positions_Ptheta)):
        if optimal_transport_plan[i, j] > 0:  # If there is mass being moved
            ax[1].arrow(positions_Pr[i], 0, positions_Ptheta[j]-positions_Pr[i], optimal_transport_plan[i, j],
                        head_width=0.05, head_length=0.1, fc='k', ec='k',
                        length_includes_head=True, alpha=0.5)
ax[1].set_title('Optimal Transport Plan')
ax[1].set_xlabel('Position')
ax[1].set_ylabel('Normalized Mass')

# Plot the final distribution P_theta with increased bar width
ax[2].bar(positions_Ptheta, masses_Ptheta, width=0.3, color='green', label='P_theta')
ax[2].set_title('Final Distribution P_theta')
ax[2].set_xlabel('Position')
ax[2].set_ylabel('Normalized Mass')

# Show the plots
plt.tight_layout()
plt.show()

# Print the Wasserstein distance
print(f"The Wasserstein distance is {wasserstein_distance:.2f}")

## Example 3

Earthmover's Distance optimization using LP Solver 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import linprog

# Function to choose text color for contrast
def get_text_color(cell_value, max_value):
    return "black" if cell_value < max_value / 2 else "white"

# Redefine the positions and masses of the distributions for Pr and Ptheta with new data
positions_Pr = np.array([0, 1, 2, 3])
masses_Pr = np.array([2, 0.5, 1, 3])
positions_Ptheta = np.array([5, 6, 7, 8])
masses_Ptheta = np.array([5, 1, 2, 3])


# Plot the orginal distributions
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].bar(positions_Pr, masses_Pr, width=0.3, color='blue', label='P_r')
ax[0].set_title('Initial Distribution P_r')
ax[0].set_xlabel('Position')
ax[0].set_ylabel('Mass')
ax[1].bar(positions_Ptheta, masses_Ptheta, width=0.3, color='green', label='P_theta')
ax[1].set_title('Final Distribution P_theta')
ax[1].set_xlabel('Position')
ax[1].set_ylabel('Mass')
plt.show()


# Normalize the masses so that they sum up to the same total mass
total_mass_Pr = np.sum(masses_Pr)
total_mass_Ptheta = np.sum(masses_Ptheta)
masses_Pr = masses_Pr / total_mass_Pr
masses_Ptheta = masses_Ptheta / total_mass_Ptheta
# print("Normalized Masses for Pr: ",masses_Pr,"\nNormalized Masses for Ptheta: ",masses_Ptheta)

# Create the cost matrix based on the distances between positions
cost_matrix = np.abs(positions_Pr[:, None] - positions_Ptheta[None, :])

# Plot the cost matrix
# Define the extent of the matrix so that the cells are centered from 0 to the number of positions
extent = [min(positions_Ptheta)-0.5, max(positions_Ptheta)+0.5, min(positions_Pr)-0.5, max(positions_Pr)+0.5]
fig, ax = plt.subplots(figsize=(6, 6))
im = ax.imshow(cost_matrix, cmap='Blues', origin='lower', extent=extent)
ax.set_title('Cost Matrix')
ax.set_xlabel('Ptheta')
ax.set_ylabel('Pr')
ax.figure.colorbar(im, ax=ax, label='Cost')
# Add text annotations
max_value_cost = np.max(cost_matrix)
for i in range(len(positions_Pr)):
    for j in range(len(positions_Ptheta)):
        ax.text(j + min(positions_Ptheta), i + min(positions_Pr), 
                f"{cost_matrix[i, j]:.1f}",
                ha="center", va="center", 
                color=get_text_color(cost_matrix[i, j], max_value_cost))
plt.show()


# Linear programming to compute the optimal transport plan
# Number of sources and targets
num_sources = len(positions_Pr)
num_targets = len(positions_Ptheta)
# Flatten the cost matrix to get the cost for each potential path
c = cost_matrix.flatten()
# Initialize the equality constraint matrix with zeros
A_eq = np.zeros((num_sources + num_targets, num_sources * num_targets))
# print("Equality Constraints: ",A_eq)
# Set up the equality constraints for the sources
for i in range(num_sources):
    A_eq[i, i * num_targets:(i + 1) * num_targets] = 1
    # print("For i = ", i, " the Equality Constraints is ",A_eq)
# Set up the equality constraints for the targets
for j in range(num_targets):
    A_eq[num_sources + j, j::num_targets] = 1
    # print("For j = ", j, " the Equality Constraints is ",A_eq)
# Concatenate the source masses and target masses for the constraints
b_eq = np.concatenate([masses_Pr, masses_Ptheta])
# Define the bounds for each variable, which is the transport amount
bounds = [(0, None)] * len(c)
# 0  indicates that the variables cannot be negative [Real valued non-negative]
# None indicates that there is no upper bound on the variables



# Solve the linear programming problem
result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')
# 'c' is the vectorised (flattened) cost matrix
# 'A_eq' is the equality constraint matrix [LHS of the equality constraints] 
# 'b_eq' is the equality constraint vector [RHS of the equality constraints]
# 'bounds' is the bounds for each variable
# 'highs' is a high performance linear programming solver
# print("Cost Matrix: ",c)
# print("Equality Constraints: ",A_eq)
# print("Equality Constraints: ",b_eq)
# print("Bounds: ",bounds)
# print("Result: ",result)

# Reshape the result to get the optimal transport plan
optimal_transport_plan = result.x.reshape(num_sources, num_targets)
# print("Optimal Transport Plan: ",optimal_transport_plan)

# Plot the heat map of the optimal transport plan
fig, ax = plt.subplots(figsize=(6, 6))
im = ax.imshow(optimal_transport_plan, cmap='Blues', origin='lower', extent=extent)
ax.set_title('Optimal Transport Plan')
ax.set_xlabel('Ptheta')
ax.set_ylabel('Pr')
ax.figure.colorbar(im, ax=ax, label='Mass to be moved from Pr to Ptheta')
# Add text annotations
max_value_optimal = np.max(optimal_transport_plan)
for i in range(len(positions_Pr)):
    for j in range(len(positions_Ptheta)):
        ax.text(j + min(positions_Ptheta), i + min(positions_Pr), 
                f"{optimal_transport_plan[i, j]:.2f}",
                ha="center", va="center", 
                color=get_text_color(optimal_transport_plan[i, j], max_value_optimal))
plt.show()


# Calculate the Wasserstein distance
wasserstein_distance = np.sum(optimal_transport_plan * cost_matrix)



## Plot the initial and final distributions and the optimal transport plan with thicker bars
fig, ax = plt.subplots(1, 3, figsize=(18, 6))

# Plot the initial distribution P_r with increased bar width
ax[0].bar(positions_Pr, masses_Pr, width=0.3, color='blue', label='P_r')
ax[0].set_title('Initial Distribution P_r')
ax[0].set_xlabel('Position')
ax[0].set_ylabel('Normalized Mass')

# Plot the optimal transport plan with arrows
for i in range(len(positions_Pr)):
    for j in range(len(positions_Ptheta)):
        if optimal_transport_plan[i, j] > 0:  # If there is mass being moved
            ax[1].arrow(positions_Pr[i], 0, positions_Ptheta[j]-positions_Pr[i], optimal_transport_plan[i, j],
                        head_width=0.05, head_length=0.1, fc='k', ec='k',
                        length_includes_head=True, alpha=0.5)
ax[1].set_title('Optimal Transport Plan')
ax[1].set_xlabel('Position')
ax[1].set_ylabel('Normalized Mass')

# Plot the final distribution P_theta with increased bar width
ax[2].bar(positions_Ptheta, masses_Ptheta, width=0.3, color='green', label='P_theta')
ax[2].set_title('Final Distribution P_theta')
ax[2].set_xlabel('Position')
ax[2].set_ylabel('Normalized Mass')

# Show the plots
plt.tight_layout()
plt.show()

# Print the Wasserstein distance
print(f"The Wasserstein distance is {wasserstein_distance:.2f}")